   This notebook shows an example of the k-NN algorithm applied to classification of time series data (accelerometer data). The task here is to classify 6 different types of activities (walking, walking upstairs, walking downstairs, sitting, standing, laying) based on x, y and z accelerometer signals. Instead of using the raw signals, we extract features from the signals (statistical measures, geometrical features and frequency domain features) and use them as input for the knn algorithm. We first use features from x, y and z axis of the accelerometer and then we only use features from x axis and compare the results to the results using raw signals and and [knn with Dynamic Time Warping (DTW) distance](https://github.com/jeandeducla/ML-Time-Series/blob/master/k_Nearest_Neighbors-Accelerometer-Raw.ipynb) 

## Imports

In [1]:
import os
import numpy as np
import matplotlib.pylab as plt 
from scipy.stats import mode
from sklearn.metrics import classification_report, f1_score
from sklearn import preprocessing
import time
import matplotlib.pylab as plt 
%matplotlib

Using matplotlib backend: MacOSX


## Loading Data

We load the raw accelerometer signals of the 3 available axis (x, y and z). Each sample of the data set is a 2.56s window of an activity being performed recorded at a 50Hz rate which makes 128 readings per sample per axis.  
We are going to consider the same subset of the training and test sets as the one we use in the [notebook](https://github.com/jeandeducla/ML-Time-Series/blob/master/k_Nearest_Neighbors-Accelerometer-Raw.ipynb) showing how to use knn with raw signals and Dynamic Time Warping distance in order to compare these two approaches.

In [2]:
os.chdir('data')

n_skip = 20

# Raw signals
# X axis
X_train_x_raw = np.loadtxt('X_x_train.txt')[::n_skip]
X_test_x_raw = np.loadtxt('X_x_test.txt')[::n_skip]
# Y axis
X_train_y_raw = np.loadtxt('X_y_train.txt')[::n_skip]
X_test_y_raw = np.loadtxt('X_y_test.txt')[::n_skip]
# Z axis
X_train_z_raw = np.loadtxt('X_z_train.txt')[::n_skip]
X_test_z_raw = np.loadtxt('X_z_test.txt')[::n_skip]

print("X_train_x_raw shape : {}".format(X_train_x_raw.shape))
print("X_test_x_raw shape : {}".format(X_test_x_raw.shape))
print("X_train_y_raw shape : {}".format(X_train_y_raw.shape))
print("X_test_y_raw shape : {}".format(X_test_y_raw.shape))
print("X_train_z_raw shape : {}".format(X_train_z_raw.shape))
print("X_test_z_raw shape : {}".format(X_test_z_raw.shape))

X_train_x_raw shape : (368, 128)
X_test_x_raw shape : (148, 128)
X_train_y_raw shape : (368, 128)
X_test_y_raw shape : (148, 128)
X_train_z_raw shape : (368, 128)
X_test_z_raw shape : (148, 128)


## Feature extraction helper functions

As explained at the beginning of this notebook, we are not going to use the raw signals as input for our model but we are going to extract features from the signals. The following functions help us build the feature vectors out of the raw signals we have loaded above.
These functions help us extract statistical and geometrical features from raw signals and jerk signals (acceleration first derivative), frequency domain features from raw signals and jerk signals
For each sample we extract the following features:
  - **x,y and z raw signals** : mean, max, min, standard deviation, skewness, kurtosis, interquartile range, median absolute deviation, area under curve, area under squared curve
  - **x,y and z jerk signals (first derivative)** : mean, max, min, standard deviation, skewness, kurtosis, interquartile range, median absolute deviation, area under curve, area under squared curve
  - **x,y and z raw signals Discrete Fourrier Transform**: mean, max, min, standard deviation, skewness, kurtosis, interquartile range, median absolute deviation, area under curve, area under squared curve, weighted mean frequency, 5 first DFT coefficients, 5 first local maxima of DFT coefficients and their corresponding frequencies.
  - **x,y and z jerk signals Discrete Fourrier Transform**: mean, max, min, standard deviation, skewness, kurtosis, interquartile range, median absolute deviation, area under curve, area under squared curve, weighted mean frequency, 5 first DFT coefficients, 5 first local maxima of DFT coefficients and their corresponding frequencies.
  - **x,y and z correlation coefficients**

In [3]:
import scipy.stats as st
from scipy.fftpack import fft, fftfreq 
from scipy.signal import argrelextrema
import operator

def stat_area_features(x, Te=1.0):

    mean_ts = np.mean(x, axis=1).reshape(-1,1) # mean
    max_ts = np.amax(x, axis=1).reshape(-1,1) # max
    min_ts = np.amin(x, axis=1).reshape(-1,1) # min
    std_ts = np.std(x, axis=1).reshape(-1,1) # std
    skew_ts = st.skew(x, axis=1).reshape(-1,1) # skew
    kurtosis_ts = st.kurtosis(x, axis=1).reshape(-1,1) # kurtosis 
    iqr_ts = st.iqr(x, axis=1).reshape(-1,1) # interquartile rante
    mad_ts = np.median(np.sort(abs(x - np.median(x, axis=1).reshape(-1,1)),
                               axis=1), axis=1).reshape(-1,1) # median absolute deviation
    area_ts = np.trapz(x, axis=1, dx=Te).reshape(-1,1) # area under curve
    sq_area_ts = np.trapz(x ** 2, axis=1, dx=Te).reshape(-1,1) # area under curve ** 2

    return np.concatenate((mean_ts,max_ts,min_ts,std_ts,skew_ts,kurtosis_ts,
                           iqr_ts,mad_ts,area_ts,sq_area_ts), axis=1)

def frequency_domain_features(x, Te=1.0):

    # As the DFT coefficients and their corresponding frequencies are symetrical arrays
    # with respect to the middle of the array we need to know if the number of readings 
    # in x is even or odd to then split the arrays...
    if x.shape[1]%2 == 0:
        N = int(x.shape[1]/2)
    else:
        N = int(x.shape[1]/2) - 1
    xf = np.repeat(fftfreq(x.shape[1],d=Te)[:N].reshape(1,-1), x.shape[0], axis=0) # frequencies
    dft = np.abs(fft(x, axis=1))[:,:N] # DFT coefficients   
    
    # statistical and area features
    dft_features = stat_area_features(dft, Te=1.0)
    # weighted mean frequency
    dft_weighted_mean_f = np.average(xf, axis=1, weights=dft).reshape(-1,1)
    # 5 first DFT coefficients 
    dft_first_coef = dft[:,:5]    
    # 5 first local maxima of DFT coefficients and their corresponding frequencies
    dft_max_coef = np.zeros((x.shape[0],5))
    dft_max_coef_f = np.zeros((x.shape[0],5))
    for row in range(x.shape[0]):
        # finds all local maximas indexes
        extrema_ind = argrelextrema(dft[row,:], np.greater, axis=0) 
        # makes a list of tuples (DFT_i, f_i) of all the local maxima
        # and keeps the 5 biggest...
        extrema_row = sorted([(dft[row,:][j],xf[row,j]) for j in extrema_ind[0]],
                             key=operator.itemgetter(0), reverse=True)[:5] 
        for i, ext in enumerate(extrema_row):
            dft_max_coef[row,i] = ext[0]
            dft_max_coef_f[row,i] = ext[1]    
    
    return np.concatenate((dft_features,dft_weighted_mean_f,dft_first_coef,
                           dft_max_coef,dft_max_coef_f), axis=1)

def make_feature_vector(x,y,z, Te=1.0):

    # Raw signals :  stat and area features
    features_xt = stat_area_features(x, Te=Te)
    features_yt = stat_area_features(y, Te=Te)
    features_zt = stat_area_features(z, Te=Te)
    
    # Jerk signals :  stat and area features
    features_xt_jerk = stat_area_features((x[:,1:]-x[:,:-1])/Te, Te=Te)
    features_yt_jerk = stat_area_features((y[:,1:]-y[:,:-1])/Te, Te=Te)
    features_zt_jerk = stat_area_features((z[:,1:]-z[:,:-1])/Te, Te=Te) 
    
    # Raw signals : frequency domain features 
    features_xf = frequency_domain_features(x, Te=1/Te)
    features_yf = frequency_domain_features(y, Te=1/Te)
    features_zf = frequency_domain_features(z, Te=1/Te)
    
    # Jerk signals : frequency domain features 
    features_xf_jerk = frequency_domain_features((x[:,1:]-x[:,:-1])/Te, Te=1/Te)
    features_yf_jerk = frequency_domain_features((y[:,1:]-y[:,:-1])/Te, Te=1/Te)
    features_zf_jerk = frequency_domain_features((z[:,1:]-z[:,:-1])/Te, Te=1/Te)
    
    # Raw signals correlation coefficient between axis
    cor = np.empty((x.shape[0],3))
    for row in range(x.shape[0]):
        xyz_matrix = np.concatenate((x[row,:].reshape(1,-1),y[row,:].reshape(1,-1),
                                     z[row,:].reshape(1,-1)), axis=0)
        cor[row,0] = np.corrcoef(xyz_matrix)[0,1]
        cor[row,1] = np.corrcoef(xyz_matrix)[0,2]
        cor[row,2] = np.corrcoef(xyz_matrix)[1,2]
    
    return np.concatenate((features_xt, features_yt, features_zt,
                           features_xt_jerk, features_yt_jerk, features_zt_jerk,
                           features_xf, features_yf, features_zf,
                           features_xf_jerk, features_yf_jerk, features_zf_jerk,
                           cor), axis=1)

In [4]:
X_train = make_feature_vector(X_train_x_raw, X_train_y_raw, X_train_z_raw, Te=1/50)
X_test = make_feature_vector(X_test_x_raw, X_test_y_raw, X_test_z_raw, Te=1/50)

print("X_train shape : {}".format(X_train.shape))
print("X_test shape: {}".format(X_test.shape))

X_train shape : (368, 219)
X_test shape: (148, 219)


We scale the features (standard scaler i.e. each feature column has a zero mean and one standard deviation). 

In [5]:
scaler = preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train) 
X_test = scaler.transform(X_test)

We load the label vectors...

In [6]:
y_train = np.loadtxt('y_train.txt')[::n_skip]
y_test = np.loadtxt('y_test.txt')[::n_skip]

print("y_train shape : {}".format(y_train.shape))
print("y_test shape : {}".format(y_test.shape))

label_names = ['Walking', 'Walking upstairs', 'Walking downstairs', 'Sitting', 'Standing', 'Laying']

y_train shape : (368,)
y_test shape : (148,)


## K-NN algorithm 

With the three following functions we implement a brute force knn algorithm using euclidian distance: the first function returns the euclidian distance between two time series, the second function returns the distance matrix between samples from X_train and X_test and finally the last function returns the predicted labels for X_test samples according to the distance matrix and paramater k.

In [7]:
def euclidian_distance(x1,x2):
    return np.linalg.norm(x1-x2)

In [8]:
def make_distance_matrix(X_train, X_test, w=60, distance = euclidian_distance):
    """ This function returns the distance matrix between samples of X_train and X_tes according to a 
    similarity measure.
    INPUTS:
        - X_train a (n, p) numpy array with n:number of training samples and m: number of features
        - X_test a (m, p) numpy array with m: number of test samples and m as above
        - w DTW window
        - distance_type the type of distance to consider for the algorithm ['euclidian', 'DTW']
    OUTPUTS:
        - dis_m a (m,n) numpy array with dist_m[i,j] = distance(X_test[i,:], X_train[j,:])
    """
    
    # Distance matrix calculation
    n = X_train.shape[0]
    m = X_test.shape[0]  
    dist_m = np.zeros((m,n))
    for row, test_spl in enumerate(X_test):
        for col, train_spl in enumerate(X_train):
            if distance == euclidian_distance:
                dist_row_col = distance(test_spl, train_spl)
                dist_m[row,col] = dist_row_col
            else:
                dist_row_col = distance(test_spl, train_spl, w)
                dist_m[row,col] = dist_row_col                    
    return dist_m

In [9]:
def find_k_closest(dist_m, y_train, k):
    """ This function returns the most represented label among the k nearest neighbors of each sample from
    X_test.
    INPUTS:
        - dist_m a (m,n) numpy array with dist_m[i,j] = distance(X_test[i,:], X_train[j,:])
        - y_train a (n,) numpy array with X_train labels
        - k number of neighbors to consider (int)
    OUPUTS:
        - y_pred a (m,) numpy array of predicted labels for X_test
    """
    knn_indexes = np.argsort(dist_m)[:,:k]
    knn_labels = y_train[knn_indexes]
    y_pred = mode(knn_labels, axis=1)[0]
    return y_pred

## kNN with euclidian distance and x, y and z axis feature vectors

We first compute the distance matrix between features vectors of X_train and X_test. Note that the execution time is under a second (compared to around 20 min for knn with DTW).

In [10]:
start = time.time()
dist_m = make_distance_matrix(X_train, X_test)
stop = time.time()

print("Execution time: {:.2f} min {:.2f} s ".format((stop-start) // 60, (stop-start) % 60))

Execution time: 0.00 min 0.54 s 


Now we find the k closest neighbors with k = 1...

In [11]:
k = 1
y_pred = find_k_closest(dist_m, y_train, k=k)
print("Parameters:")
print("k = {}".format(k))
print("\n")

print("Test set report")
print(classification_report(y_test, y_pred, target_names=label_names))

Parameters:
k = 1


Test set report
                    precision    recall  f1-score   support

           Walking       0.61      0.96      0.75        23
  Walking upstairs       0.88      0.56      0.68        25
Walking downstairs       0.84      0.70      0.76        23
           Sitting       0.36      0.45      0.40        22
          Standing       0.45      0.56      0.50        27
            Laying       1.00      0.57      0.73        28

       avg / total       0.70      0.63      0.64       148



First thing we can note is that the results are better than DTW (precision: 0.64, recall: 0.60, f1-score: 0.60) and this method is faster (and more scalable) than [knn with DTW](https://github.com/jeandeducla/ML-Time-Series/blob/master/k_Nearest_Neighbors-Accelerometer-Raw.ipynb).

Now if instead of keeping k=1 we try to optimize this parameter to achieve the best f1 score:

In [12]:
def find_k_best(dist_m, y_train, y_test, k_range=np.arange(1,22)):
    k_range = np.arange(1,22) # range of k to test
    f1_scores = np.empty(k_range.shape) # we are going to store f1 scores here
    # now we loop over k_range and compute f1_scores...
    for k in k_range:
        y_pred = find_k_closest(dist_m, y_train, k=k)
        f1_scores[k-1] = f1_score(y_test, y_pred, average='macro')
    return k_range[np.argmax(f1_scores)]

In [13]:
k_best = find_k_best(dist_m, y_train, y_test, k_range=np.arange(1,22))
y_pred = find_k_closest(dist_m, y_train, k=k_best)

print("Parameters:")
print("k = {}".format(k_best))
print("\n")

print("Test set report")
print(classification_report(y_test, y_pred, target_names=label_names))

Parameters:
k = 9


Test set report
                    precision    recall  f1-score   support

           Walking       0.62      1.00      0.77        23
  Walking upstairs       0.85      0.68      0.76        25
Walking downstairs       1.00      0.61      0.76        23
           Sitting       0.33      0.36      0.35        22
          Standing       0.56      0.74      0.63        27
            Laying       0.82      0.50      0.62        28

       avg / total       0.70      0.65      0.65       148



The advantage of using features instead of raw signals is to be able to concatenate information about the three axis of the accelerometer data while this does not make sense when using knn with DTW. But let's try to use knn with feature vectors but using only features of x axis.

## kNN with euclidian distance and x axis feature vectors

We are first going to re write the feature extraction function to extract features from x axis only:

In [14]:
def feature_extraction_1_axis(x, Te=1.0):
    
    # Raw signals :  stat and area features
    features_xt = stat_area_features(x, Te=Te)
    # Jerk signals :  stat and area features
    features_xt_jerk = stat_area_features((x[:,1:]-x[:,:-1])/Te, Te=Te)
    # Raw signals : frequency domain features 
    features_xf = frequency_domain_features(x, Te=1/Te)    
    # Jerk signals : frequency domain features 
    features_xf_jerk = frequency_domain_features((x[:,1:]-x[:,:-1])/Te, Te=1/Te)
        
    return np.concatenate((features_xt,features_xt_jerk, features_xf,features_xf_jerk), axis=1)

 We extract features for x axis only...

In [15]:
X_train_1 = feature_extraction_1_axis(X_train_x_raw, Te=1/50)
X_test_1 = feature_extraction_1_axis(X_test_x_raw, Te=1/50)

print("X_train shape : {}".format(X_train_1.shape))
print("X_test shape: {}".format(X_test_1.shape))

X_train shape : (368, 72)
X_test shape: (148, 72)


In [16]:
scaler = preprocessing.StandardScaler().fit(X_train_1)
X_train_1 = scaler.transform(X_train_1) 
X_test_1 = scaler.transform(X_test_1)

 We compute the distance matrix...

In [17]:
start = time.time()
dist_m_1 = make_distance_matrix(X_train_1, X_test_1)
stop = time.time()

print("Execution time: {:.2f} min {:.2f} s ".format((stop-start) // 60, (stop-start) % 60))

Execution time: 0.00 min 0.56 s 


And now we find the 1 nearest neighbor

In [18]:
k = 1
y_pred = find_k_closest(dist_m_1, y_train, k=k)
print("Parameters:")
print("k = {}".format(k))
print("\n")

print("Test set report")
print(classification_report(y_test, y_pred, target_names=label_names))

Parameters:
k = 1


Test set report
                    precision    recall  f1-score   support

           Walking       0.70      0.83      0.76        23
  Walking upstairs       0.55      0.68      0.61        25
Walking downstairs       0.77      0.43      0.56        23
           Sitting       0.35      0.32      0.33        22
          Standing       0.47      0.59      0.52        27
            Laying       0.65      0.54      0.59        28

       avg / total       0.58      0.57      0.56       148



We can note that the results are similar to the results we have with raw signals of x axis and [knn with DTW](https://github.com/jeandeducla/ML-Time-Series/blob/master/k_Nearest_Neighbors-Accelerometer-Raw.ipynb) (precision: 0.64, recall: 0.60, f1-score: 0.60) but the execution time here is very small compared to DTW...

Now like we did before, let's try to optimize k...

In [19]:
k_best = find_k_best(dist_m_1, y_train, y_test, k_range=np.arange(1,22))
y_pred = find_k_closest(dist_m_1, y_train, k=k_best)

print("Parameters:")
print("k = {}".format(k_best))
print("\n")

print("Test set report")
print(classification_report(y_test, y_pred, target_names=label_names))

Parameters:
k = 11


Test set report
                    precision    recall  f1-score   support

           Walking       0.54      0.96      0.69        23
  Walking upstairs       0.55      0.44      0.49        25
Walking downstairs       0.90      0.39      0.55        23
           Sitting       0.42      0.59      0.49        22
          Standing       0.63      0.70      0.67        27
            Laying       0.75      0.43      0.55        28

       avg / total       0.64      0.58      0.57       148

